### Activation Patching on Parallel Corpora
We pass the same sentence in different languages to the model, to see differing activations


In [17]:
import transformer_lens
import plotly.io as pio
import circuitsvis as cv
import torch
import torch.nn as nn
import tqdm.auto as tqdm
from functools import partial
import einops
from fancy_einsum import einsum 
import plotly.express as px 
from jaxtyping import Float
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, FactoredMatrix

pio.renderers.default = "png"
torch.set_grad_enabled(False)

# Testing that the library works
cv.examples.hello("Ajay")

device = utils.get_device()
print("Device:", device)

Device: mps


In [18]:
model_name = "meta-llama/Llama-3.1-8B" #"bigscience/bloom-560m"
model = HookedTransformer.from_pretrained(model_name, device=device)

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.hf.co/repos/59/43/594310891758e12c04540e5fb01589e425e25052bfa08d42a457c38d95b552b6/c28b25e7541751056ee126627e007f8d4288319733285e9f7b17b9ff6eb313f0?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00002-of-00004.safetensors%3B+filename%3D%22model-00002-of-00004.safetensors%22%3B&Expires=1732890816&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMjg5MDgxNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzU5LzQzLzU5NDMxMDg5MTc1OGUxMmMwNDU0MGU1ZmIwMTU4OWU0MjVlMjUwNTJiZmEwOGQ0MmE0NTdjMzhkOTViNTUyYjYvYzI4YjI1ZTc1NDE3NTEwNTZlZTEyNjYyN2UwMDdmOGQ0Mjg4MzE5NzMzMjg1ZTlmN2IxN2I5ZmY2ZWIzMTNmMD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=YY26COdDyl4Pw-gveOr8nxtOjASGkPoQWrKaAIDXT95DPm77aIrCer2HQlBDXQOYC70jNrTSo4k4bU9FA6V0RncRxjp8iN4hmB61-TUUDvnV3nfrOU7t9uXuOOIQPnixfn7eY7jXSuMJ6gqMbgQxBfRx%7ER8lHZZleD-ccOPj2FO3dlL6C8ZF1u7UpakYJ8tztm5sDnI1gIpJVm5jY5WMVm3Ggy1idVwCuL7-

...

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

RuntimeError: MPS backend out of memory (MPS allocated: 18.12 GB, other allocations: 896.00 KB, max allowed: 18.13 GB). Tried to allocate 64.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [4]:
from datasets import load_dataset

language_pair = "en-es" # unfortunately, for this huggingface impl, only english-nonenglish sets are available!
ds = load_dataset("sentence-transformers/parallel-sentences-global-voices", language_pair)
data = ds["train"]
print("language_pair=", language_pair)
print("number of examples=", len(data))


language_pair= en-es
number of examples= 248152


In [5]:
for i in range(3):
    print("=========",i,"=========")
    str_en = data[i]["english"]
    str_es = data[i]["non_english"]
    loss_en = model(str_en, return_type="loss")
    loss_es = model(str_es, return_type="loss")
    print("str_en=", str_en)
    print("loss_en=", loss_en)
    print("str_es=", str_es)
    print("loss_es=", loss_es)

========= 0 =========
str_en= Argentina: Stencil Graffiti · Global Voices
loss_en= tensor(6.3151, device='mps:0')
str_es= Argentina: Graffitis
loss_es= tensor(6.6812, device='mps:0')
========= 1 =========
str_en= Line of Sight has an excellent collection of stencil graffiti from around Buenos Aires.
loss_en= tensor(4.8579, device='mps:0')
str_es= 'Line of Sight' tiene una excelente colección de graffitis de lugares de Buenos Aires.
loss_es= tensor(4.4299, device='mps:0')
========= 2 =========
str_en= Most images have captions explaining the significance.
loss_en= tensor(5.9069, device='mps:0')
str_es= La mayoría de las imágenes tienen notas explicando el significado.
loss_es= tensor(4.9595, device='mps:0')


In [8]:
str_en = data[1]["english"]
str_es = data[1]["non_english"]
tokens_en = model.to_tokens(str_en)
tokens_es = model.to_tokens(str_es)
model_logits, model_cache = model.run_with_cache(tokens_en, remove_batch_dim=True)
print(type(model_cache))
attention_pattern_en = model_cache["pattern", 0, "attn"]
print(attention_pattern_en.shape)
str_tokens_en = model.to_str_tokens(str_en)

print("Layer 0 Head Attention Patterns(EN):")
cv.attention.attention_patterns(tokens=str_tokens_en, attention=attention_pattern_en)

<class 'transformer_lens.ActivationCache.ActivationCache'>
torch.Size([16, 19, 19])
Layer 0 Head Attention Patterns(EN):


In [9]:
model_logits, model_cache = model.run_with_cache(tokens_es, remove_batch_dim=True)
print(type(model_cache))
attention_pattern_es = model_cache["pattern", 0, "attn"]
print(attention_pattern_es.shape)
str_tokens_es = model.to_str_tokens(str_es)

print("Layer 0 Head Attention Patterns (ES):")
cv.attention.attention_patterns(tokens=str_tokens_es, attention=attention_pattern_es)

<class 'transformer_lens.ActivationCache.ActivationCache'>
torch.Size([16, 20, 20])
Layer 0 Head Attention Patterns (ES):


In [12]:
for key in model_cache.keys():
    print (key, model_cache[key].shape)

embed.ln.hook_scale torch.Size([20, 1])
embed.ln.hook_normalized torch.Size([20, 1024])
hook_embed torch.Size([20, 1024])
blocks.0.hook_resid_pre torch.Size([20, 1024])
blocks.0.ln1.hook_scale torch.Size([20, 1])
blocks.0.ln1.hook_normalized torch.Size([20, 1024])
blocks.0.attn.hook_q torch.Size([20, 16, 64])
blocks.0.attn.hook_k torch.Size([20, 16, 64])
blocks.0.attn.hook_v torch.Size([20, 16, 64])
blocks.0.attn.hook_attn_scores torch.Size([16, 20, 20])
blocks.0.attn.hook_pattern torch.Size([16, 20, 20])
blocks.0.attn.hook_z torch.Size([20, 16, 64])
blocks.0.hook_attn_out torch.Size([20, 1024])
blocks.0.hook_resid_mid torch.Size([20, 1024])
blocks.0.ln2.hook_scale torch.Size([20, 1])
blocks.0.ln2.hook_normalized torch.Size([20, 1024])
blocks.0.mlp.hook_pre torch.Size([20, 4096])
blocks.0.mlp.hook_post torch.Size([20, 4096])
blocks.0.hook_mlp_out torch.Size([20, 1024])
blocks.0.hook_resid_post torch.Size([20, 1024])
blocks.1.hook_resid_pre torch.Size([20, 1024])
blocks.1.ln1.hook_scale